# Installations / Préparations

## install

In [23]:
!pip install importlib-metadata==4.13.0

In [24]:
!pip install git+https://github.com/osigaud/bbrl_gym

  Cloning https://github.com/osigaud/bbrl_gym to c:\users\29530\appdata\local\temp\pip-req-build-105jwcq3
  Resolved https://github.com/osigaud/bbrl_gym to commit 5557075ecd7d4171ac0c21be3c69a94bcae655a9
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/osigaud/bbrl_gym 'C:\Users\29530\AppData\Local\Temp\pip-req-build-105jwcq3'


## import

In [25]:
import os
from typing import Tuple, List

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from mazemdp.toolbox import egreedy, egreedy_loc
from mazemdp.maze_plotter import show_videos
from mazemdp.mdp import Mdp
from bbrl_gym.envs.maze_mdp import MazeMDPEnv

# For visualization
os.environ["VIDEO_FPS"] = "5"
if not os.path.isdir("./videos"):
    os.mkdir("./videos")

from IPython.display import Video

import gym
import bbrl_gym

## settings

In [26]:
# Settings
NB_EPISODES = 100
TIMEOUT = 100
MAZE_LARGEUR = 4
MAZE_HAUTEUR = 3
EPSILON = 0.02

## création du maze

In [27]:
env = gym.make("MazeMDP-v0", kwargs={"width": MAZE_LARGEUR, "height": MAZE_HAUTEUR, "ratio": 0.25})
env.reset()

# in dynamic programming, there is no agent moving in the environment
env.init_draw("The maze")

Output()

# Q-Learning 3D

## utils

In [28]:
def calcul_transition(mdp: MazeMDPEnv, but):
    transition_matrix = np.zeros( (mdp.nb_states, mdp.action_space.n, mdp.nb_states) )

    for s in range(mdp.nb_states-1):
        x = mdp.coord_x[s]
        y = mdp.coord_y[s]

        # NORD #
        # si case tout au nord, pas de changement d'état possible dans cette direction
        if x == 0 :
            transition_matrix[s][0][s] = 1.0
        # SUD #
        # si case tout au sud, pas de changement d'état possible dans cette direction
        if x == MAZE_HAUTEUR-1 :
            transition_matrix[s][1][s] = 1.0
        # EST #
        # si case tout à l'est, pas de changement d'état possible dans cette direction
        if y == MAZE_LARGEUR-1 :
            transition_matrix[s][2][s] = 1.0
        # OUEST
        # si case tout à l'ouest, pas de changement d'état possible dans cette direction
        if y == 0 :
            transition_matrix[s][3][s] = 1.0

        for s_prime in range(mdp.nb_states-1):
            # Regarde les relations entre s et les autres états
            if (s != s_prime):
                x_prime = mdp.coord_x[s_prime]
                y_prime = mdp.coord_y[s_prime]
                # NORD #
                if (x_prime == x-1) and (y_prime == y):
                    transition_matrix[s][0][s_prime] = 1.0
                # SUD #
                if (x_prime == x+1) and (y_prime == y):
                    transition_matrix[s][1][s_prime] = 1.0
                # EST #
                if (x_prime == x) and (y_prime == y+1):
                    transition_matrix[s][2][s_prime] = 1.0
                # OUEST #
                if (x_prime == x) and (y_prime == y-1):
                    transition_matrix[s][3][s_prime] = 1.0

        # Si une action n'a aucune transition, alors on boucle sur le même état
        for a in range(mdp.action_space.n):
            if all(transition_matrix[s,a] == 0):
                transition_matrix[s,a,s] = 1.0

    # Transition Matrix of terminal states
    transition_matrix[but, :, :] = 0
    transition_matrix[but, :, -1] = 1

    return transition_matrix

In [29]:
def get_policy_from_q(Q: np.ndarray, but: int) -> np.ndarray:
    # Outputs a policy given the action values
    policy = np.argmax(Q[:, but, :], axis=1)
    return policy

In [30]:
# calcul et stocke les matrices de récompense et de transition pour un but
def calcul_goal(mdp: MazeMDPEnv, but: int):
    # calcul du nouveau point d'origine des récompenses
    new_r = np.zeros((mdp.nb_states, mdp.action_space.n))
    new_r[but] = np.ones(mdp.action_space.n)
    new_r[but] = np.ones(mdp.action_space.n)

    # calcul de la nouvelle matrice de transition
    new_P = calcul_transition(mdp, but)

    return new_r, new_P

In [31]:
# modifie la MDP en fonction du goal
def maj_goal(mdp: MazeMDPEnv, but, r_list, P_list):
    # modification de l'état but
    mdp.mdp.r = r_list[but]
    mdp.P = P_list[but]
    mdp.mdp.P = P_list[but]

## epsilon-greedy

In [32]:
# --------------------------- Q-Learning epsilon-greedy version -------------------------------#

# Given an exploration rate epsilon, the QLearning algorithm computes the state action-value function
# based on an epsilon-greedy policy
# alpha is the learning rate


def q_learning_eps(
    mdp: MazeMDPEnv,
    epsilon: float,
    nb_episodes: int = 20,
    timeout: int = 50,
    alpha: float = 0.5,
    render: bool = True,
) -> Tuple[np.ndarray]:
    # Initialize the state-goal-action value function
    # 3D : états / buts / actions
    Q = np.zeros((mdp.nb_states, mdp.nb_states, mdp.action_space.n))
    r_list = []   # récompenses selon les buts
    P_list = []   # transition selon les buts

    # calcul initial des matrices pour chacun des buts
    for s in range(mdp.nb_states-1):
        new_r, new_P = calcul_goal(mdp, s)
        r_list.append(new_r)
        P_list.append(new_P)
  
    # Run learning cycle
    mdp.set_timeout(timeout)  # episode length

    for _ in tqdm(range(nb_episodes)):
        # Draw the first state of episode i using a uniform distribution over all the states
        s = mdp.reset(uniform=True)
        but = np.random.randint(mdp.nb_states-1)   # tirage de l'état but (hors état puit)
        #print("but : ", but)      # aide visualisation

        maj_goal(mdp, but, r_list, P_list)    # modification des matrices correspondante à ce but

        if render:
            mdp.init_draw("Q-learning e-greedy"+str(but))

        done = mdp.mdp.done()

        while not done:
            if render:
                # Show the agent in the maze
                mdp.mdp.plotter.terminal_states = [but]
                mdp.draw_v_pi(Q[:,but,:], get_policy_from_q(Q, but))

            # Draw an action using an epsilon-greedy policy
            a = egreedy(Q[:, but, :], s, epsilon)

            # Perform a step of the MDP
            [s_prime, r, done, _] = mdp.step(a)

            # Calculs en fonction du but de l'épisode
            delta = r + mdp.gamma * np.max(Q[s_prime, but]) - Q[s, but, a]

            Q[s, but, a] += alpha * delta

            # Update the agent position
            s = s_prime


    if render:
        # Show the final policy
        for but in tqdm(range(mdp.nb_states-1)):
            mdp.mdp.plotter.terminal_states = [but]   # modifie le plotter
            mdp.draw_v_pi(Q[:,but,:], get_policy_from_q(Q, but), title="Q-learning e-greedy"+str(but))

    return Q

In [33]:
Q = q_learning_eps(env, EPSILON, NB_EPISODES, TIMEOUT, render=False)

  0%|          | 0/100 [00:00<?, ?it/s]

In [34]:
# visualisation
for but in tqdm(range(env.nb_states-1)):
    print("état but : ", but)
    env.mdp.plotter.terminal_states = [but]
    env.draw_v_pi(Q[:,but,:], get_policy_from_q(Q, but), title="Q-learning e-greedy"+str(but))

  0%|          | 0/9 [00:00<?, ?it/s]

état but :  0


Output()

état but :  1


état but :  2


...

état but :  4


état but :  5


état but :  6


état but :  7


état but :  8


In [38]:
from mazemdp.toolbox import softmax, sample_categorical

In [39]:
# --------------------------- Q-Learning softmax version ----------------------------#

# Given a temperature "tau", the QLearning algorithm computes the state action-value function
# based on a softmax policy
# alpha is the learning rate


def q_learning_soft(
    mdp: MazeMDPEnv,
    tau: float,
    nb_episodes: int = 20,
    timeout: int = 50,
    alpha: float = 0.5,
    render: bool = True,
) -> Tuple[np.ndarray, List[float], List[int]]:
    # Initialize the state-action value function
    # alpha is the learning rate
    Q = np.zeros((mdp.nb_states,mdp.nb_states, mdp.action_space.n))

    r_list = []
    P_list =[]

    # calcul initial des matrices pour chacun des buts
    for s in range(mdp.nb_states-1):
        new_r, new_P = calcul_goal(mdp, s)
        r_list.append(new_r)
        P_list.append(new_P)
    
 
    # Run learning cycle
    mdp.set_timeout = timeout  # episode length

    

    for episode in tqdm(range(nb_episodes)):
        # Draw the first state of episode i using a uniform distribution over all the states
        x = mdp.reset(uniform=True)
        but = np.random.randint(0,mdp.nb_states-1)
        print("but : ", but)

        maj_goal(mdp, but, r_list, P_list)

       
        t = tau / (1 + np.log(1+episode))

        if render:
            mdp.init_draw("Q-learning softmax"+str(but))

        done = mdp.mdp.done()
        while not done:
            if render:
                # Show the agent in the maze
                mdp.draw_v_pi(Q[:,but,:], get_policy_from_q(Q, but))

            
            u = sample_categorical(softmax(Q[:, but, :], x, t))
            # Perform a step of the MDP
            [y, r, done, _] = mdp.step(u)

            # Update the state-action value function
            delta = r + mdp.gamma * np.max(Q[y, but]) - Q[x, but, u]
            Q[x,but, u] = Q[x,but, u] + alpha * delta
            
            x = y


    if render:
         # Show the final policy
        for but in tqdm(range(mdp.nb_states-1)):
            mdp.current_state = 0
            mdp.terminal_states = [but]
            mdp.draw_v_pi(Q[:,but,:], get_policy_from_q(Q, but), title="Q-learning softmax"+str(but))
    return Q, r_list


In [41]:
tau = 0.1
q, r_list = q_learning_soft(env, tau,render= False)

  0%|          | 0/20 [00:00<?, ?it/s]

but :  4
but :  6
but :  6
but :  7
but :  8
but :  7
but :  4
but :  4
but :  1
but :  1
but :  2
but :  7
but :  1
but :  7
but :  1
but :  3
but :  1
but :  6
but :  5
but :  3


In [42]:
# visualisation
for but in tqdm(range(env.nb_states-1)):
    print("état but : ", but)
    env.mdp.plotter.terminal_states = [but]
    env.draw_v_pi(q[:,but,:], get_policy_from_q(q, but), title="Q-learning e-greedy"+str(but))

  0%|          | 0/9 [00:00<?, ?it/s]

état but :  0


Output()

état but :  1


état but :  2


...

état but :  4


état but :  5


état but :  6


état but :  7


état but :  8
